In [1]:
# Big Multiple AGN Catalog (Big MAC) DR1 - ``Offset AGN'' Candidates (== Dual SMBH Candidates)
# Author: R. W. Pfeifle
# Original Date Created: 10 Sept. 2020
# Last Revision Date: 2 Oct. 2020

# New Form Creation Date: 13 January 2022
# Last Revision: 16 February 2024

# Purpose: Combine various catalogs of double-peaked optically selected dual AGN candidates
# 

In [2]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [3]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [4]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
    return tunique, tmatches, idx1, idx2





In [5]:
# tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,smith2012,5)

# # Adding the DOI, author, and bibcode info to all of the Smith+2010 rows here in the matches table...
# for index, row in tmatches.iterrows():
#     if row['Table_flag']!='Table2':
#         tmatches.at[index, 'Paper(s)'] += ' ; Smith+2012'
#         tmatches.at[index, 'BibCode(s)'] += ' ; 2012ApJ...752...63S' 
#         tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/752/1/63'

# # Now clipping out all Smith+2010 rows from the matches table
# tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)

# # Concatenating everything together to generate a master table here
# the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
# the_whills.drop(labels=['index','Table_flag'], axis=1, inplace=True) #'level_0',

# tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,fu2011d,3)

# #len(tmatches)

# # This finds 16 matches, which is exactly what we'd expect (there are 16 doubles in the table from Fu+2011)

# for i, j in zip(idx1, idx2):
#     #print("i:", i, "j:", j)
#     #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
#     #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
#     the_whills.at[i, 'Sep'] = fu2011d.at[j, 'Sep_as']
#     the_whills.at[i, 'Paper(s)'] += ' ; Fu+2011'
#     the_whills.at[i, 'BibCode(s)'] += ' ; 2011ApJ...733..103F' 
#     the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/733/2/103'
#     the_whills.at[i, 'dV'] = fu2011d.at[j, 'dV']
#     the_whills.at[i, 'Notes'] += ' Fu+2011 companions within 3 arcseconds,'

# for index, row in the_whills.iterrows():
#     if index in idx1:
#         the_whills.at[index, 'Paper(s)'] += ' ; Fu+2012'
#         the_whills.at[index, 'BibCode(s)'] += ' ; 2012ApJ...745...67F' 
#         the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/745/1/67'
#         the_whills.at[index, 'Notes'] += ' Companion(s) within 3 arcseconds.'





In [6]:
# here we're going too load in the 'lost' offset AGN candidates from Comerford+2009
# we will NOT be matching it to the tables in this notebook. Instead we will match it directly with the main \
# table in the matching catalogs notebook

# Here we're loading in the double-peaked emission line galaxy catalog of comerford2009
comerford2009 = pd.read_csv('Tables/comerford2009/Comerford2009_offsetagns.csv', sep=',')

comerford2009['Name1'] = comerford2009['ID']
comerford2009['Name2'] = "-99"
comerford2009['z1'] = comerford2009['zabs']
comerford2009['z2'] = -99
comerford2009['z1_type'] = "spec"
comerford2009['z2_type'] = "-99"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(comerford2009,comerford2009['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = comerford2009['RA1'], dec = comerford2009['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
comerford2009['RA1_deg'] = coordconvert.ra.degree
comerford2009['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
comerford2009['RA2'] = comerford2009['RA1']
comerford2009['Dec2'] = comerford2009['Dec1']

# just duplicates of RA1 for now. We will correc this later....
comerford2009['RA2_deg'] = comerford2009['RA1_deg']
comerford2009['Dec2_deg'] = comerford2009['Dec1_deg']

# Adding details about the coordinates
comerford2009['Equinox1'] = "J2000"
comerford2009['Coordinate_waveband1'] = "Optical"
comerford2009['Coordinate_Source1'] = "EGSD2"

comerford2009['System Type'] = 'Dual SMBH Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
comerford2009['Brightness1'] = -99
comerford2009['Brightness_band1'] = "-99"
comerford2009['Brightness_type1'] = "-99"

comerford2009['Brightness2'] = -99
comerford2009['Brightness_band2'] = "-99"
comerford2009['Brightness_type2'] = "-99"

comerford2009['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
comerford2009['dV'] = comerford2009['vem-vabs(kms-1)']
## dV here is ...


# Adding information about the paper and the selection method
comerford2009['Selection Method'] = "Optical Spectroscopy / Slit Optical Spectroscopy / Velocity Offset Narrow Optical Spectroscopic Emission Lines" #DPSELs
comerford2009['Confirmation Method'] = "-99"
comerford2009['Paper(s)'] = "Comerford+2009a"
comerford2009['BibCode(s)'] = "2009ApJ...698..956C"
comerford2009['DOI(s)'] = "https://doi.org/10.1088/0004-637X/698/1/956"

cols_to_drop = ['ID','U-B','MB(-5logh)','[Oiii]λ5007/Hβ','zabs','zem','vem-vabs(kms-1)','vem-vabs(kms-1)_pm']

for i in cols_to_drop:
    try:
        comerford2009.drop(labels=str(i), axis=1, inplace=True)
    except:
        print('This column is not available in this version. Must have modified another notebook.')

comerford2009.to_csv('Comerford2009_offsettargets.csv', sep=',', index=False)

#comerford2009





In [7]:
# Here we're loading in the double-peaked emission line galaxy catalog of Comerford2013
comerford2013 = ((Table.read('Tables/Comerford2013/table1.dat', readme = 'Tables/Comerford2013/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---','HbVoff','e_HbVoff','O3Voff','e_O3Voff','HaVoff','e_HaVoff','O3/Hb','e_O3/Hb','N2/Ha','e_N2/Ha','S2/Ha','e_S2/Ha','O1/Ha','e_O1/Ha','Class','n_Class'])
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Comerford2013 looked at 173 Type 2 AGNs in Deep2 and found only two double-peaked dual AGN candidates \
# and found five offset AGNs. 

NDWFS = ['J143044.06+335224.5','J143053.69+345836.4','J143316.48+353259.3','J143317.07+344912.0','J143710.03+343530.1']
comerford2013 = comerford2013[comerford2013['NDWFS'].isin(NDWFS)]

comerford2013['Coordinates'] = comerford2013['NDWFS'].str.slice(start=1)
comerford2013['RA_test'] = comerford2013['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
comerford2013['Dec_test'] = comerford2013['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
comerford2013['RA1'] = comerford2013['RA_test'].str.slice(start=0, stop=2)+":"+comerford2013['RA_test'].str.slice(start=2, stop=4)+":"+comerford2013['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
comerford2013['Dec1'] = comerford2013['Dec_test'].str.slice(start=0, stop=3)+":"+comerford2013['Dec_test'].str.slice(start=3, stop=5)+":"+comerford2013['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
comerford2013.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)


comerford2013['Name1'] = comerford2013['NDWFS']
comerford2013['Name2'] = "-99"
comerford2013['z1'] = comerford2013['z']
comerford2013['z2'] = -99
comerford2013['z1_type'] = "spec"
comerford2013['z2_type'] = "-99"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(comerford2013,comerford2013['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = comerford2013['RA1'], dec = comerford2013['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
comerford2013['RA1_deg'] = coordconvert.ra.degree
comerford2013['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
comerford2013['RA2'] = comerford2013['RA1']
comerford2013['Dec2'] = comerford2013['Dec1']

comerford2013['RA2_deg'] = comerford2013['RA1_deg']
comerford2013['Dec2_deg'] = comerford2013['Dec1_deg']

# Adding details about the coordinates
comerford2013['Equinox1'] = "J2000"
comerford2013['Coordinate_waveband1'] = "Optical"
comerford2013['Coordinate_Source1'] = "AGES"

comerford2013['System Type'] = 'Dual SMBH Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
comerford2013['Brightness1'] = -99
comerford2013['Brightness_band1'] = "-99"
comerford2013['Brightness_type1'] = "-99"

comerford2013['Brightness2'] = -99
comerford2013['Brightness_band2'] = "-99"
comerford2013['Brightness_type2'] = "-99"

comerford2013['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
comerford2013['dV'] = -99
for index, row in comerford2013.iterrows():
    if row['NDWFS']=='J143044.06+335224.5':
        comerford2013.at[index, 'dV'] = 217.7
    if row['NDWFS']=='J143053.69+345836.4':
        comerford2013.at[index, 'dV'] = -123.4
    if row['NDWFS']=='J143316.48+353259.3':
        comerford2013.at[index, 'dV'] = -100.9
    if row['NDWFS']=='J143317.07+344912.0':
        comerford2013.at[index, 'dV'] = -143.0
    if row['NDWFS']=='J143710.03+343530.1':
        comerford2013.at[index, 'dV'] = -97.6
# dV here is the weighted mean velocity offste measured from the Balmer and forbidden lines


# Adding information about the paper and the selection method
comerford2013['Selection Method'] = "Optical Spectroscopy / Fiber Spectroscopy / Velocity Offset Narrow Optical Spectroscopic Emission Lines" #DPSELs
comerford2013['Confirmation Method'] = "-99"
comerford2013['Paper(s)'] = "Comerford+2013"
comerford2013['BibCode(s)'] = "2013ApJ...777...64C"
comerford2013['DOI(s)'] = "https://doi.org/10.1088/0004-637X/777/1/64"

#comerford2013

In [8]:
# Loading in the tables from Comerford+2014 here now...

comerford2014 = ((Table.read('Tables/Comerford2014/table1.dat', readme = 'Tables/Comerford2014/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---'])

comerford2014['Name1'] = comerford2014['SDSS']
comerford2014['Name2'] = "-99"
comerford2014['z1'] = comerford2014['z']
comerford2014['z2'] = -99
comerford2014['z1_type'] = "spec"
comerford2014['z2_type'] = "-99"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(comerford2014,comerford2014['SDSS'])

# Converting the coordinates
comerford2014['RA1'] = comerford2014['RA']
comerford2014['Dec1'] = comerford2014['Dec']

coordconvert = SkyCoord(ra = comerford2014['RA1'], dec = comerford2014['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
comerford2014['RA1_deg'] = coordconvert.ra.degree
comerford2014['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
comerford2014['RA2'] = comerford2014['RA']
comerford2014['Dec2'] = comerford2014['Dec']

comerford2014['RA2_deg'] = comerford2014['RA1_deg']
comerford2014['Dec2_deg'] = comerford2014['Dec1_deg']

# Adding details about the coordinates
comerford2014['Equinox1'] = "J2000"
comerford2014['Coordinate_waveband1'] = "Optical"
comerford2014['Coordinate_Source1'] = "SDSS"

comerford2014['System Type'] = 'Dual SMBH Candidate'
# these objects could technically also be recoil candidates

# Adding in some columns that we'll population via a Simbad or Ned search later
comerford2014['Brightness1'] = -100
comerford2014['Brightness_band1'] = "-100"
comerford2014['Brightness_type1'] = "-100"

comerford2014['Brightness2'] = -100
comerford2014['Brightness_band2'] = "-100"
comerford2014['Brightness_type2'] = "-100"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
comerford2014['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#comerford2014['Sep(kpc)'] = comerford2014['Sep']*((cosmo.arcsec_per_kpc_proper(comerford2014['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
comerford2014['dV'] = comerford2014['DelVW'] 
# dV here is taken from the weighted average of the velocity offsets measured for the Balmer and the forbidden lines

# Adding information about the paper and the selection method
comerford2014['Selection Method'] = "Optical Spectroscopy / Fiber Spectroscopy / Velocity Offset Narrow Optical Spectroscopic Emission Lines" #DPSELs
comerford2014['Confirmation Method'] = "-99"
comerford2014['Paper(s)'] = "Comerford+2014"
comerford2014['BibCode(s)'] = "2014ApJ...789..112C"
comerford2014['DOI(s)'] = "https://doi.org/10.1088/0004-637X/789/2/112"

comerford2014['Notes'] = ''


#comerford2014




In [9]:
tunique, tmatches, idx1, idx2 = match_tables_fib(comerford2013,comerford2014,25)

# Apparently no matches to within 20''! 
print(len(tmatches))

# concatenating now...

the_whills = pd.concat([comerford2013,comerford2014])
the_whills.reset_index(drop=True, inplace=True)
# Verified



0


In [10]:
# Adding in information from Allen+2015

allenobjs = ['J120401.97+012641.6','J011333.06+002947.9'] # Allen+ used these: J120401.97+012641.5 J011333.05+002948.0

# Adding the DOI, author, and bibcode info to all of the Comerford+2014 rows here in the matches table...
for index, row in the_whills.iterrows():
    if row['Name1'] in allenobjs:
        the_whills.at[index, 'Paper(s)'] += ' ; Allen+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015MNRAS.451.2780A' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stv1121'

allenobjs = ['J120401.97+012641.6']
for index, row in the_whills.iterrows():
    if row['Name1'] in allenobjs:
        the_whills.at[index, 'Notes']+=' Allen+ use IFS observations to show that the offset emission line(s) used to flag this target originally is not the result of a binary system but due to large-scale motions of gas in the galaxy. If this system does constitute a merger they cannot rule out the possible presence of two SMBHs. There is a suggestion that it could be a merger but no firm evidence. '
        the_whills.at[index, 'System Type'] = 'Likely Single AGN'

allenobjs = ['J011333.06+002947.9']
for index, row in the_whills.iterrows():
    if row['Name1'] in allenobjs:
        the_whills.at[index, 'Notes']+=' Allen+ uses IFS observations to show that the offset emission line(s) are likely due to an outflow rather than the motion of a NLR. They argue strongly against a binary scenario.'
        the_whills.at[index, 'System Type'] = 'Likely Single AGN'

#the_whills

# verified that this matching works properly

In [11]:
# Adding in the information from Barrows+

barrowsobjs = ['J111519.97+542316.6'] #J111519.98+542316.65

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if row['Name1'] in barrowsobjs:
        the_whills.at[index, 'Paper(s)'] += ' ; Barrows+2016'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016ApJ...829...37B' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/829/1/37'
        #the_whills.at[index, 'System Type']='Dual AGN'
        the_whills.at[index, 'Notes']+=' Selected by Barrows+ as an offset AGN candidate.'
        #the_whills.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'
    


In [12]:
# Adding in information from Muller-Sanchez+2016

# these were Muller-Sanchez's original names:
#msobjs = ['J101847.57+294114.10','J105553.64+152027.40','J111729.21+614015.30','J134640.79+522836.60']
msobjs = ['J101847.57+294114.1','J105553.64+152027.5','J111729.22+614015.2','J134640.79+522836.5']
    
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if row['Name1'] in msobjs:
        the_whills.at[index, 'Paper(s)'] += ' ; Müller-Sánchez+2016'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016ApJ...830...50M' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/830/1/50'
        #the_whills.at[index, 'Notes']+='AGNs confirmed via X-rays and reanalyses of optical spectroscopy.'



#msobjs = ['J101847.57+294114.10','J105553.64+152027.40','J134640.79+522836.60']
msobjs = ['J101847.57+294114.1','J105553.64+152027.5','J134640.79+522836.5']
for index, row in the_whills.iterrows():
    if row['Name1'] in msobjs:
        the_whills.at[index, 'Notes']+=' Muller-Sanchez+2016 used OSIRIS IFS observations to show that the observed spatial distribution and kinematics of the ionized gas indicate the presence of outflows and are spatially offset from the center of the galaxy. The spatially offset peaks in the emission can be explained via shocks. The integrated OSIRIS spectrum shows the same velocity offset as observed in SDSS but the spectral profiles are different; the near-IR lines show clear asymmetric wings not seen in the optical.'

msobjs = ['J111729.22+614015.2'] 
for index, row in the_whills.iterrows():
    if row['Name1'] in msobjs:
        the_whills.at[index, 'Notes']+=' Muller-Sanchez+2016 used OSIRIS IFS observations to show a counter-rotating circumnuclear disk contains the peak of the offset Pa-alpha emission. The most plausible origin of this enhanced emission is that it lies at an intersection zone between the nuclear disk and bar of the galaxy where cloud-cloud collisions drive the emission. The integrated OSIRIS spectrum shows the same velocity offset as observed in SDSS.'



# Verified that this matches properly


In [13]:
#the_whills

In [14]:
# We need to add in any missing tables from Comerford+ and all of the Barrows+ tables

# I think we're missiing comerford2009? And maybe another one post-2010








In [15]:
# here we're going to load in and format the Barrows+2016 table
# NOTE: we have removed the objects already flagged in other tables

# Namely:
#J080523.29+281815.8
#J090714.45+520343.4
#J105842.44+314457.6
#J111519.23+542310.9
#
#J111458.01+403611.4
#134442.1781
#J110851.04+065901.4
#J123420.14+475155.9

barrows2016 = pd.read_csv('Tables/Barrows2016/Barrows2016.csv', sep=',')

# here we're going too load in the 'lost' offset AGN candidates from Barrows+
# we will NOT be matching it to the tables in this notebook. Instead we will match it directly with the main \
# table in the matching catalogs notebook


barrows2016['Name1'] = barrows2016['Name1']
barrows2016['Name2'] = "-99"
barrows2016['z1'] = barrows2016['z1']
barrows2016['z2'] = -99
barrows2016['z1_type'] = "spec"
barrows2016['z2_type'] = "-99"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(barrows2016,barrows2016['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = barrows2016['RA1'], dec = barrows2016['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
barrows2016['RA1_deg'] = coordconvert.ra.degree
barrows2016['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
barrows2016['RA2'] = barrows2016['RA1']
barrows2016['Dec2'] = barrows2016['Dec1']

# just duplicates of RA1 for now. We will correc this later....
barrows2016['RA2_deg'] = barrows2016['RA1_deg']
barrows2016['Dec2_deg'] = barrows2016['Dec1_deg']

# Adding details about the coordinates
barrows2016['Equinox1'] = "J2000"
barrows2016['Coordinate_waveband1'] = "Optical"
barrows2016['Coordinate_Source1'] = "SDSS"

barrows2016['System Type'] = 'Dual SMBH Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
barrows2016['Brightness1'] = -99
barrows2016['Brightness_band1'] = "-99"
barrows2016['Brightness_type1'] = "-99"

barrows2016['Brightness2'] = -99
barrows2016['Brightness_band2'] = "-99"
barrows2016['Brightness_type2'] = "-99"

barrows2016['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#barrows2016['dV'] = barrows2016['vem-vabs(kms-1)']
#for index, row in barrows2016.iterrows():
#    if row['NDWFS']=='J143044.06+335224.5':
#        barrows2016.at[index, 'dV'] = 217.7
#    if row['NDWFS']=='J143053.69+345836.4':
#        barrows2016.at[index, 'dV'] = -123.4
#    if row['NDWFS']=='J143316.48+353259.3':
#        barrows2016.at[index, 'dV'] = -100.9
#    if row['NDWFS']=='J143317.07+344912.0':
#        barrows2016.at[index, 'dV'] = -143.0
#    if row['NDWFS']=='J143710.03+343530.1':
#        barrows2016.at[index, 'dV'] = -97.6
## dV here is the weighted mean velocity offste measured from the Balmer and forbidden lines


# Adding information about the paper and the selection method
barrows2016['Selection Method'] = "Optical Spectroscopy / Optical Fiber Spectroscopy / Optical Spectroscopic Emission Line Ratios / X-ray Imaging / X-ray Positional Offset" #DPSELs
barrows2016['Confirmation Method'] = "-99"
barrows2016['Paper(s)'] = "Barrows+2016"
barrows2016['BibCode(s)'] = "2016ApJ...829...37B"
barrows2016['DOI(s)'] = "https://doi.org/10.3847/0004-637X/829/1/37"

barrows2016.to_csv('barrows2016_offsettargets.csv', sep=',', index=False)



In [16]:
#the_whills

In [17]:
# now formatting the table...
the_whills.drop(labels=['index','NDWFS','z','Table_flag','SDSS','e_z','DelVB','e_DelVB','DelVF','e_DelVF',\
                        'DelVW','e_DelVW'], axis=1, inplace=True)



In [18]:
# now saving the table...
the_whills.to_csv('Offset_AGN_tables_DR1.csv', sep=',', index=False)

